In [11]:
import time
import re
from bs4 import BeautifulSoup #use beautifulsoup to scrape data
from selenium import webdriver


url = "http://millercenter.org/the-presidency/presidential-speeches"

browser = webdriver.Chrome() #change this line depending on your system
browser.get(url) #go to the website 


SCROLL_PAUSE_TIME = 0.8


#slight changes to be made!!

# Get scroll height
last_height = browser.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to bottom
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = browser.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

soup = BeautifulSoup(browser.page_source, 'lxml')


In [12]:
import urllib3 
import pandas as pd
from datetime import datetime
import time
import string
import numpy as np



link_str = []

for link in soup.findAll('a', href=True):
    if link != None:
        link_str.append(link['href'])

# now I want the link strings containting /the-presidency/
new_link_str = []

for element in link_str:
    if element.find("/presidential-speeches/") >= 0: 
        new_link_str.append(element)
        


http = urllib3.PoolManager()
list2=[]
president_names=[]
for element in new_link_str:
    url = "http://millercenter.org" + element
#     print(element)
    response = http.request('GET', url)
    soup_2 = BeautifulSoup(response.data, 'lxml')
    list2.append(soup_2.findAll('div', {"class":"transcript-inner"}))
    president_names.append(soup_2.find('p', {"class":"president-name"}).get_text(strip=True))
    


#converting into string due to weird beautifulsoup output    
list3=[]
for i in range(len(list2)):
    if list2[i]:
        list3.append(str(list2[i]))
    else:
        
        #add comments for why I had to do it like this
        
        url = "http://millercenter.org" + new_link_str[i]
        response = http.request('GET', url)
        soup_2 = BeautifulSoup(response.data, 'lxml')
        list3.append(str(soup_2.findAll('div', {"class":"view-transcript"})))

# print(list3[1])

#this will be used to clean the text from html tags
def tag_cleanr(text):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', text)
    return cleantext

cleaned_speeches = []

#these replace lines could and should be replaced with regex since we covered that in the syllabus more extensively

for i in range(len(list3)):
    x = tag_cleanr(list3[i])
    x = x.replace('Transcript', '').replace('\r\n', '').replace('\n', '')
    cleaned_speeches.append(x)


#Get all the speech dates into a list

pattern = r'(\w+-\d+-\d{4})'
speech_dates=[]


for i in range(len(new_link_str)):
    match = re.findall(pattern, new_link_str[i])
    speech_dates.append(str(match))


for i in range(len(speech_dates)):
    #this feels like its terrible syntax but not sure if there is a nicer way, maybe I will replace it with regex
    
    speech_dates[i] = speech_dates[i].replace('[', '').replace(']', ''.replace("'", "")).replace("'", "")
    speech_dates[i] = datetime.strptime(speech_dates[i], '%B-%d-%Y')


df = pd.DataFrame({'dates':speech_dates, 'speeches':cleaned_speeches, 'president':president_names})
df = df.set_index('dates')

#this part is added because dataframe does not recognize [] as an empty input. 

df['speeches'].replace('[]', np.nan, inplace=True)

#now that I replaced [] with NaN, I can simply say drop all the rows with NaN elements

df.dropna(subset=['speeches'], inplace=True)

df.truncate(after=pd.Timestamp('1901-09-14')).tail()

print(df)



                                                     speeches  \
dates                                                           
2018-03-19  [THE PRESIDENT: Thank you to our First Lady, M...   
2018-02-23  [THE PRESIDENT: Thank you very much. Thank you...   
2018-02-15  [THE PRESIDENT: My fellow Americans, today I s...   
2018-02-01  [THE PRESIDENT: Thank you, Paul and Mitch, for...   
2018-01-30  [Mr. Speaker, Mr. Vice President, Members of C...   
2018-01-26  [PRESIDENT TRUMP: Thank you, Klaus, very much....   
2017-12-18  [THE PRESIDENT: Thank you very much. Thank you...   
2017-09-19  [PRESIDENT TRUMP: Mr. Secretary General, Mr. P...   
2017-07-24  [TRUMP: Thank you, everybody. Thank you very m...   
2017-06-29  [THE PRESIDENT: Thank you, everybody. Thank yo...   
2017-02-28  [Mr. Speaker, Mr. Vice President, Members of C...   
2017-01-20  [Chief Justice Roberts, President Carter, Pres...   
2016-05-15  [Hello Rutgers!  (Applause.)  R-U rah-rah!  (A...   
2016-03-22  [Thank you.  